## **Extracción de Características Auditivas (Acoustic Embeddings)**

En este notebook abordamos la fase de Feature Extraction para la modalidad de audio. A diferencia de la transcripción de texto (donde importa qué se dice), en el análisis acústico nos centramos en cómo se dice. Para capturar esta información paralingüística, aplicaremos dos estrategias, extrayendo dos tipos de características complementarias, siguiendo un flujo de trabajo similar al notebook **3.4_a**:

1. Representaciones de Aprendizaje Profundo (Deep Learning): **Wav2Vec 2.0**.
    * Este modelo, desarrollado por Facebook AI, aprende representaciones contextuales directamente de la forma de onda del audio sin procesar. Basado en Transformers, utiliza mecanismos de auto-atención (self-attention) para capturar dependencias temporales complejas. Al haber sido pre-entrenado en miles de horas de audio no etiquetado, genera embeddings que encapsulan información fonética, prosódica y emocional de alta calidad. Es el estándar de facto (SOTA). Se ha seleccionado el modelo **Base** (768) en lugar de **Large** (1024) ya que, al igual que el vídeo, los audios en MELD e IEMOCAP son muy cortos, por tanto un vector de 1024 dimensiones hubiera memorizado el ruido de fondo o particularidades del locutor en lugar de características emocionales generales. La versión **Base** ofrece una representación más generalista y robusta para la detección de estrés.

        - **SALIDA**: Vectores contextuales de **768** dimensiones.

2. Características *Artesanales* (*Hand-crafted Features*): **MFCCs + Energía**.
    * Como línea base (baseline) robusta y computacionalmente ligera, extraemos características espectrales y prosódicas utilizando la librería `librosa`. 
    Los **MFCCs** (**Mel-frequency cepstral coefficients**) son coeficientes que representan el timbre de la voz tal y como lo percibe el oído humano. Son el estándar histórico en el reconocimiento de emociones. La **Energía** (**RMS**) y **ZCR** (*Zero-Crossing Rate*) son indicadores directos de la intensidad sonora (volumen) y la velocidad/"rugosidad" del habla, variables que suelen aumentar en estados de estrés.
        - **MFCCs**: Extraeremos **13** coeficientes. Se seleccionan los primeros 13 ya que modelan eficazmente la envolvente del tracto vocal, descartando el ruido de alta frecuencia.
        - **Energía** (**RMS**): La raíz cuadrática media (*Root Mean Square*) es un indicador directo de la intensidad sonora. Responde a la pregunta de: *¿El sujeto grita o susurra?*. Extraeremos **1 RMS**.
        - **ZCR**: La tasa de cruces por cero mide la frecuencia de cambio de signo en la señal. Responde a la pregunta de: *¿Es una voz limpia, o hay ruido/rugosidad?*. Extraeremos **1 ZCR**.
    
        - **SALIDA**: Vectores de **15** dimensiones (13+1+1).

A diferencia de la extracción de vídeo (donde fijamos un número fijo de **32 frames**), en audio, la dimensión temporal (*Time Steps*) será variable en función de la duración de cada frase. Con **Wav2Vec 2.0** generamos un vector de características cada 20 ms aproximadamente, con dimensiones $(T_w, 768)$.Con la alternativa *Hand-crafted* generamos un vector de características en función del tamaño de ventana y salto (*hop length*) definidos, típicamente cada 32 ms, con dimensiones de $(T_h, 15)$.



In [ ]:
!pip install -q transformers librosa 

In [1]:
# Carga previa de todas las librerías y paquetes necesarios

import os
import shutil
import zipfile
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from tqdm import tqdm  # Para barra de progreso
from google.colab import drive
import random
from transformers import logging as hf_logging

# Silencionamos los warnings:
hf_logging.set_verbosity_error()

# Configuración de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo: {device}")

Dispositivo: cuda


In [2]:
# Se monta Drive para acceder a los datos y guardar los resultados:
drive.mount('/content/drive')

Mounted at /content/drive


Traemos los datos al disco del servidor de Colab.

In [ ]:
# Rutas de origen en Drive:
ZIP_PATH_IEMOCAP = '/content/drive/MyDrive/Proyecto_TFG_Data/IEMOCAP_Audio.zip'
ZIP_PATH_MELD = '/content/drive/MyDrive/Proyecto_TFG_Data/MELD_Audio.zip'
CSV_PATH = '/content/drive/MyDrive/Proyecto_TFG_Data/Multimodal_Stress_Dataset.csv'

# Rutas locales en Colab (Entorno local)
LOCAL_DATA_ROOT = '/content/data'
IEMOCAP_AUDIO_PATH = os.path.join(LOCAL_DATA_ROOT, 'IEMOCAP_Audio')

# Rutas de SALIDA en Drive
OUTPUT_DIR = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_AUDIO'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Copiamos y descomprimimos los archivos ZIP con los datos en el almacenamiento local de Colab (más rápido que trabajar directamente desde Drive):
if not os.path.exists(LOCAL_DATA_ROOT):
    os.makedirs(LOCAL_DATA_ROOT, exist_ok=True)
    
    # ------------ MELD --------
    if os.path.exists(ZIP_PATH_MELD):
        print(f"Copiando {ZIP_PATH_MELD} a {LOCAL_DATA_ROOT}.")
        shutil.copy(ZIP_PATH_MELD, LOCAL_DATA_ROOT)
        with zipfile.ZipFile(os.path.join(LOCAL_DATA_ROOT, 'MELD_Audio.zip'), 'r') as z:
            z.extractall(LOCAL_DATA_ROOT)
            
    # ------------ IEMOCAP ---------
    if os.path.exists(ZIP_PATH_IEMOCAP):
        print(f"Copiando {ZIP_PATH_IEMOCAP} a {LOCAL_DATA_ROOT}.")
        shutil.copy(ZIP_PATH_IEMOCAP, LOCAL_DATA_ROOT)
        with zipfile.ZipFile(os.path.join(LOCAL_DATA_ROOT, 'IEMOCAP_Audio.zip'), 'r') as z:
            z.extractall(LOCAL_DATA_ROOT)
else:
    print(f"Los datos ya existen en {LOCAL_DATA_ROOT}.")

# Cargamos del CSV Global
if os.path.exists(CSV_PATH):
    df_global = pd.read_csv(CSV_PATH)
    display(df_global.head()) 
else:
    print(f"No se encuentra el archivo en {CSV_PATH}")

Copiando /content/drive/MyDrive/Proyecto_TFG_Data/MELD_Audio.zip a /content/data.
Copiando /content/drive/MyDrive/Proyecto_TFG_Data/IEMOCAP_Audio.zip a /content/data.


,Utterance_ID,Dialogue_ID,video_path,audio_path,Transcription,duration,split,target_stress,dataset_origin
0,train_dia0_utt0,0,train_splits/dia0_utt0.mp4,MELD_Audio/train_dia0_utt0.wav,also I was the point person on my company's tr...,5.672333,train,0,MELD
1,train_dia0_utt1,0,train_splits/dia0_utt1.mp4,MELD_Audio/train_dia0_utt1.wav,You must've had your hands full.,1.501500,train,0,MELD
2,train_dia0_utt2,0,train_splits/dia0_utt2.mp4,MELD_Audio/train_dia0_utt2.wav,That I did. That I did.,2.919583,train,0,MELD
3,train_dia0_utt3,0,train_splits/dia0_utt3.mp4,MELD_Audio/train_dia0_utt3.wav,So let's talk a little bit about your duties.,2.752750,train,0,MELD
4,train_dia0_utt4,0,train_splits/dia0_utt4.mp4,MELD_Audio/train_dia0_utt4.wav,My duties? All right.,6.464792,train,0,MELD


---

### **Análisis Previo**

A diferencia del vídeo, donde fijamos 32 frames, en audio mantendremos la **longitud variable** para no perder información prosódica importante (silencios, cambios de tono, etc.).

* Analizamos la duración de los audios para confirmar la variabilidad de los datos que alimentarán al modelo.


In [11]:
def analizar_duracion_audio(df, path_meld, path_iemocap, sample_size=50):
    """
    Selecciona un batch aleatorio por cada dataset (MELD e IEMOCAP), de acuerdo al tamaño indicado, y analiza la duración en segundos.
    - Devuelve una lista con la media, duración mínima y máxima para cada dataset.
    - Nota: Para MELD, si el audio no está extraído, se intentará leer directamente del vídeo asociado.
    """
    resultados = []
    
    for origen in ['MELD', 'IEMOCAP']:
        subset = df[df['dataset_origin'] == origen]
        rutas = subset['audio_path'].sample(n=sample_size, random_state=42).tolist()

        durations = []
        for ruta in rutas:
            # ---------- MELD ------------
            if origen == 'MELD':
                # Intentamos buscar el archivo. Si el audio está extraído o usamos el vídeo asociado
                # Nota: librosa puede leer audio directamente desde el .mp4 si el .wav no existe
                full_path = os.path.join(path_meld, ruta)
                if not os.path.exists(full_path): # Si no está el wav, probamos con el video asociado
                     video_r = subset[subset['audio_path'] == ruta]['video_path'].values[0]
                     full_path = os.path.join(path_meld, video_r)
            else:
                full_path = os.path.join(path_iemocap, ruta)
            
            if os.path.exists(full_path):
                dur = librosa.get_duration(path=full_path)
                durations.append(dur)
        
        if durations:
            resultados.append([np.mean(durations), np.min(durations), np.max(durations)])
        else:
            resultados.append([0, 0, 0])
            
    return resultados

resultados = analizar_duracion_audio(df_global, LOCAL_DATA_ROOT, IEMOCAP_AUDIO_PATH)
print(f"MELD Audio (seg): Media={resultados[0][0]:.2f}, Min={resultados[0][1]:.2f}, Max={resultados[0][2]:.2f}")
print(f"IEMOCAP Audio (seg): Media={resultados[1][0]:.2f}, Min={resultados[1][1]:.2f}, Max={resultados[1][2]:.2f}")

MELD Audio (seg): Media=3.65, Min=0.08, Max=9.63
IEMOCAP Audio (seg): Media=4.13, Min=1.16, Max=12.77


---
## **Wav2Vec 2.0**

In [12]:
# --- 1. Se carga inicialmente el modelo WAV2VEC 2.0 (960 horas) ---


def extract_wav2vec(audio_path,processor,model):
    """
    Con el modelo Wav2Vec 2.0 (960 horas) cargado previamente, extrae embeddings de audio.
    Retorna embeddings de tamaño (Time_Steps, 768).
    Sampling Rate obligatorio: 16000 Hz.
    """
    try:
        # Cargamos audio con obligatoriamente 16kHz:
        y, sr = librosa.load(audio_path, sr=16000)
        
        # Aplicamos preprocesamiento (Tokenización + Padding interno del modelo):
        inputs = processor(y, sampling_rate=16000, return_tensors="pt", padding=True)
        input_values = inputs.input_values.to(device)
        
        with torch.no_grad():
            outputs = model(input_values)
            # outputs.last_hidden_state tiene forma (1, T, 768) -> quitamos el batch con squeeze
            hidden_states = outputs.last_hidden_state.squeeze(0).cpu().numpy()
            
        return hidden_states
    except Exception as e:
        return None


---
## **MFCCs + RMS + ZCR (*Hand-crafted* baseline)**

In [ ]:
# --- 2. HAND-CRAFTED (Baseline) ---
def extract_handcrafted(audio_path):
    """
    Carga el audio, extrae 13 MFCCs, RMS y ZCR, y los apila en una matriz de características.
    Retorna matriz de (Time_Steps, 15).
    Compuesta por: 13 MFCCs + 1 RMS + 1 ZCR.
    """
    try:
        y, sr = librosa.load(audio_path, sr=16000) # sr = 16000 Hz es la frecuencia de muestreo (número de muestra/pasos de tiempo por segundo) (obligatorio para consistencia con Wav2Vec), significa que si el audio no está a esa frecuencia, se re-muestreará automáticamente
        hop_length = 512 # Aprox 32ms
        
        # -----------> 13 MFCCs (Timbre / Tracto Vocal)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=hop_length) #hop_length=512 es que cada 512 muestras/pasos de tiempo se calcula un nuevo vector de características, lo que da una resolución temporal de aproximadamente 32ms (512 muestras a 16kHz)
        
        # --------->RMS (Energía)
        rms = librosa.feature.rms(y=y, hop_length=hop_length)
        
        # ----------> ZCR (Rugosidad/Ruido)
        zcr = librosa.feature.zero_crossing_rate(y=y, hop_length=hop_length)
        
        # Apilamos: (15, T) -> Transponemos a (T, 15)
        features = np.vstack([mfcc, rms, zcr]).T
        return features
    except Exception as e:
        return None

----
## ***Feed-Forward*. Extracción de Características**

In [ ]:
# --- PROCESAMIENTO ---

# Cargamos el modelo Wav2Vec 2.0 (960 horas) una sola vez para usarlo en todo el proceso:
# Cargamos también el processor:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav2vec_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
wav2vec_model.eval()

# Definimos un directorio temporal local para ir almacenando los resultados antes de copiarlos a Drive (más rápido trabajar localmente y luego los movemos a Drive):
TEMP_LOCAL_DIR = '/content/temp_features_audio'
temp_dirs = {
    'wav2vec': os.path.join(TEMP_LOCAL_DIR, 'audio_wav2vec'),
    'handcrafted': os.path.join(TEMP_LOCAL_DIR, 'audio_handcrafted')
}
for p in temp_dirs.values(): 
    os.makedirs(p, exist_ok=True)


for index, row in tqdm(df_global.iterrows(), total=len(df_global), desc="Procesando Audio"):
    
    # 1. Definimos los nombres de salida
    fname = f"{row['Dialogue_ID']}_{row['Utterance_ID']}.npy".replace("/", "_")
    out_w2v = os.path.join(temp_dirs['wav2vec'], fname)
    out_hc = os.path.join(temp_dirs['handcrafted'], fname)
    
    # Chequeo si ya existe (para reanudar si se corta la ejecución):
    if os.path.exists(out_w2v) and os.path.exists(out_hc):
        continue

    # 2. Localizar archivo de entrada
    # Nota: Usamos la columna 'audio_path' del CSV, pero si falla, probamos con 'video_path'
    # ya que librosa puede extraer el audio del vídeo mp4 directamente.
    
    if row['dataset_origin'] == 'MELD':
        # MELD suele tener estructura compleja.
        # Intento 1: Ruta exacta del audio csv
        audio_path = os.path.join(LOCAL_DATA_ROOT, row['audio_path'])
        if not os.path.exists(audio_path):
            # Intento 2: Ruta del vídeo (librosa extraerá el audio)
            audio_path = os.path.join(LOCAL_DATA_ROOT, row['video_path'])
    else:
        # IEMOCAP
        audio_path = os.path.join(LOCAL_DATA_ROOT, row['audio_path'])
        if not os.path.exists(audio_path):
             audio_path = os.path.join(LOCAL_DATA_ROOT, row['video_path'])
    
    # 3. Procesar si existe el archivo
    if os.path.exists(audio_path):
        # Wav2Vec
        emb_w2v = extract_wav2vec(audio_path, processor, wav2vec_model)
        # Handcrafted
        emb_hc = extract_handcrafted(audio_path)
        
        if emb_w2v is not None and emb_hc is not None:
            np.save(out_w2v, emb_w2v)
            np.save(out_hc, emb_hc)

# --- SUBIDA A DRIVE ---
# Comprimimos la carpeta temporal con los resultados:
shutil.make_archive("/content/features_audio_COMPLETO", 'zip', TEMP_LOCAL_DIR)
shutil.copy("/content/features_audio_COMPLETO.zip", OUTPUT_DIR)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]config.json: 0.00B [00:00, ?B/s]tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]Loading weights:   0%|          | 0/210 [00:00<?, ?it/s]
Procesando Audio:   0%|          | 47/21219 [00:13<34:24, 10.26it/s]  /usr/local/lib/python3.12/dist-packages/librosa/core/spe

### **Sanity Check**

Verificamos finalmente para asegurar que tenemos todos los *embeddings* cargados correctamente (conteo), además de mostrar un ejemplo verificando ese tamaño variable ya mencionado.

In [ ]:
# Listar archivos generados en local
files_w2v = os.listdir(temp_dirs['wav2vec'])
files_hc = os.listdir(temp_dirs['handcrafted'])

print(f"Total esperado: {len(df_global)}")
print(f"Wav2Vec generados: {len(files_w2v)}")
print(f"Handcrafted generados: {len(files_hc)}")

# Verificar variabilidad de dimensiones (Ejemplo aleatorio)
if len(files_w2v) > 0:
    sample = random.choice(files_w2v)
    
    data_w2v = np.load(os.path.join(temp_dirs['wav2vec'], sample))
    data_hc = np.load(os.path.join(temp_dirs['handcrafted'], sample))
    
    print(f"\nEjemplo ({sample}):")
    print(f"Wav2Vec Shape: {data_w2v.shape}")
    print(f"Handcrafted Shape: {data_hc.shape}")

Total esperado: 21219
Wav2Vec generados: 21219
Handcrafted generados: 21219

Ejemplo (308_train_dia308_utt5.npy):
Wav2Vec Shape: (112, 768) 
Handcrafted Shape: (71, 15)
